# Optimization 1:
In order to optimize the model, below are the tweaks made in the datset:

- In addition to "EIN" and "NAME" columns, "SPECIAL_CONSIDERATIONS" column also dropped off before creating the model.

- Bin size of "APPLICATION_TYPE" others value increased. "T10" was added to Others.

- Binning structure of CLASSIFICATION also changed.

- More neurons were added to hidden layers.

- EPOCHS is changed from 50 to 100



## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df1 = application_df.drop(columns=['EIN','NAME','SPECIAL_CONSIDERATIONS'])
application_df1

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,142590,1
...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,5000,1


In [3]:
# Determine the number of unique values in each column.
application_df1.nunique()

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
STATUS                 2
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
value_counts= application_df1['APPLICATION_TYPE'].value_counts()
print(value_counts)

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64


In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff_value = 700
application_types_to_replace = value_counts[value_counts < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
Other      804
T8         737
T7         725
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df1['CLASSIFICATION'].value_counts
classification_counts

<bound method IndexOpsMixin.value_counts of 0        C1000
1        C2000
2        C3000
3        C2000
4        C1000
         ...  
34294    C1000
34295    C3000
34296    C2000
34297    C3000
34298    C1000
Name: CLASSIFICATION, Length: 34299, dtype: object>

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts = application_df1['CLASSIFICATION'].value_counts()
value_counts_greater_than_1 = classification_counts[classification_counts > 5]
print(value_counts_greater_than_1)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
Name: CLASSIFICATION, dtype: int64


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_cutoff_value = 778
classifications_to_replace = classification_counts[classification_counts < classification_cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
# Step 1: Identify categorical columns
#categorical_columns = ['AFFILIATION', 'USE_CASE', 'ORGANIZATION','INCOME_AMT']  # Replace with the names of your categorical columns

# Step 2: Use pd.get_dummies() to create dummy variables
#dummy_df = pd.get_dummies(application_df1[categorical_columns], drop_first=True)

# Step 3 (optional): Merge the dummy variables with the original DataFrame
#application_df1 = pd.concat([application_df1, dummy_df], axis=1)

# Step 4 (optional): Drop the original categorical columns if desired
#application_df1.drop(columns=categorical_columns, inplace=True)

application_df1 = pd.get_dummies(application_df1,dtype=float)
application_df1.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T12,APPLICATION_TYPE_T13,APPLICATION_TYPE_T14,APPLICATION_TYPE_T15,APPLICATION_TYPE_T17,APPLICATION_TYPE_T19,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6692,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,142590,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df1['IS_SUCCESSFUL'].values
X = application_df1.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42 )

<ipython-input-10-288c7069af4b>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = application_df1.drop(['IS_SUCCESSFUL'],1).values


In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  90
hidden_nodes_layer2 = 40

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, activation='relu', input_dim=number_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 90)                10350     
                                                                 
 dense_1 (Dense)             (None, 40)                3640      
                                                                 
 dense_2 (Dense)             (None, 1)                 41        
                                                                 
Total params: 14,031
Trainable params: 14,031
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 7s 5ms/step - loss: 0.5752 - accuracy: 0.7190
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5554 - accuracy: 0.7292
Epoch 3/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5508 - accuracy: 0.7307
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5497 - accuracy: 0.7317
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5479 - accuracy: 0.7309
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5464 - accuracy: 0.7329
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5458 - accuracy: 0.7315
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5442 - accuracy: 0.7339
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5431 - accuracy: 0.7333
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5429 - accura

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5729 - accuracy: 0.7263 - 471ms/epoch - 2ms/step
Loss: 0.5729121565818787, Accuracy: 0.7262973785400391


In [16]:
#Export our model to HDF5 file

from google.colab import files
!cp 'C:\Users\kanwal\Documents\UT Austin Projects\UTA-VIRT-DATA-PT-02-2023-U-LOLC\02-Homework\21-Neural-Networks-Deep-Learning\Model\AlphabetSoupCharity_Optimization1.h5'
nn.save("AlphabetSoupCharity_Optimization1.h5")
files.download('AlphabetSoupCharity_Optimization1.h5')

cp: missing destination file operand after 'C:\Users\kanwal\Documents\UT Austin Projects\UTA-VIRT-DATA-PT-02-2023-U-LOLC\02-Homework\21-Neural-Networks-Deep-Learning\Model\AlphabetSoupCharity_Optimization1.h5'
Try 'cp --help' for more information.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>